DNN testing playground! 

i wanna put all the final shit for the DNN into a .py file but for now while I fuck with stuff, ill do it here. that is if this bad boi can even handle the big ass .h5py files I made, if not either make smaller ones or just use alpha.

In [22]:
# boilerplate, ensure you have these packages
import numpy as np
import tensorflow as tf
from keras import callbacks
from keras.callbacks import LearningRateScheduler as LRS
from keras.models import Model
from keras.layers import *
from keras import optimizers as kopt
import keras.backend as K
import datetime
import h5py
import hyperopt as hopt

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11071562665701749354
]


-5.0

In [7]:
# data rescaling functions, want everything to be [0,1]
# rescaling thicknesses, angle of incid, psi, delta
# this info should be pulled from the corresponding .txt file!!!
# ^although im not quite sure how I wanna do tht rn so just list it in the .txt file
def resc_l(thickness):
    'rescales the thicknesses to [0,1], takes in arrays'
    resc = thickness*1e7 
    return resc
def resc_ang(aoi):
    'rescales the aoi to [0,1], takes in arrays in DEGREES'
    resc = aoi/90.
    return resc
def resc_psi(psi):
    '''
    rescales ellipsometric coeff psi to basically [0,1], takes in array in DEGREES. returns in rescaled degrees
    '''
    resc = psi/90.
    return resc
def resc_del(delta):
    '''
    rescales ellipsometric coeff delta to be positive (by adding phase shift if needed)
    takes in array in RADIANS (b/c its the natural output of tmm.ellips). returns rescaled degrees
    '''
    if (delta.min()<0.):
        resc = delta + np.pi
    resc = np.rad2deg(resc)/360.
    return resc

In [27]:
# funct to read in data
def read_dat(fname,dataset,nlay,nang,nwvl):
    '''
    function to read in the training data within an .h5 file and rescale
    currently used for ENZ proj with data for angle, thicknesses, rp, rs, tp, ts, psi, delta
    NOTE: data should be a [N,T] array, w/ N for num of runs and T = nang + nlay + 6*nang*nwvl
    inputs:
    fname - string - file name containing the training data, pls use full file name
    dataset - string - name of the dataset in the .h5 file
    NOTE: currently doesn't support reading in data from more than 1 dataset
    nlay - integer - number of layers in the multistack
    nang - integer - number of angles of incidence investigated
    nwvl - integer - number of wavelengths investigated
    NOTE: will throw errors if inputs are not correct
    outputs:
    ang - array, float - [N,nang] array of rescaled angles
    l - array, float - [N,nlay] array of rescaled thicknesses
    rp,rs,tp,ts - array, float - each is a [N,nang*nwvl] array of R and T params
    psi, delta - array, float - each is a [N,nang*nwvl] array of ellipsometric coeff in degrees
    '''
    # assertions to make sure inputs are good
    assert isinstance(nang,int), 'incorrect input for: nang'
    assert isinstance(nlay,int), 'incorrect input for: nlay'
    assert isinstance(nwvl,int), 'incorrect input for: nwvl'
    assert isinstance(fname,str), 'incorrect input for: fname'
    assert isinstance(dataset,str), 'incorrect input for: dataset'
    with h5py.File(fname,'r') as h:
        dat = np.array(h.get(dataset))
    # assertion to make sure this will read in data correctly
    assert (dat[0,:].size == nang+nlay+6*nang*nwvl), 'total data shape and input shape do not match'
    ang = dat[:,0:nang]
    l = dat[:,nang:(nang+nlay)]
    high = nang+nlay+nang*nwvl
    rp = dat[:,(nang+nlay):high]
    low = high
    high += nang*nwvl
    rs = dat[:,low:high]
    low = high
    high += nang*nwvl
    tp = dat[:,low:high]
    low = high
    high += nang*nwvl
    ts = dat[:,low:high]
    low = high
    high += nang*nwvl
    psi = dat[:,low:high]
    low = high
    high += nang*nwvl
    delta = dat[:,low:high]
    # assertion to make sure data was read in correctly
    assert (dat[-1,-1]==delta[-1,-1]), 'data was read incorrectly'
    # delete the original array that was read in as it is just a space waster now
    del dat
    l = resc_l(l)
    ang = resc_ang(ang)
    psi = resc_psi(psi)
    # to make sure phase shift is introduced correctly, need to seperate delta by angle
    if (nang>1):
        for i in range(nang):
            delta[:,(i*nwvl):((i+1)*nwvl)] = resc_del(delta[:,(i*nwvl):((i+1)*nwvl)])
    delta = resc_del(delta)
    return (ang,l,rp,rs,tp,ts,psi,delta)

In [26]:
a = 3
for i in range(a):
    print(i)

0
1
2
